<a href="https://colab.research.google.com/github/MaryamTavakoli/RangerAI-assignment/blob/main/RFX_Insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Installing Libraries**

In [ ]:
!pip install llama-index-llms-openai
!pip install llama-index-embeddings-openai
!pip install llama-index-readers-file
!pip install llama-index-finetuning
!pip install llama-index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nl

## **Mounting Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Loading Documents**

In [ ]:
import os
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

In [ ]:
docs_dir = '/content/drive/MyDrive/RangerAI/Documents/'

docs_paths = []
for doc_file in os.listdir(docs_dir):
    if doc_file.endswith('.pdf'):
        docs_paths.append(os.path.join(docs_dir, doc_file))

In [ ]:
def load_corpus(files, verbose=False):
    if verbose:
        print(f"Loading files {files}")

    reader = SimpleDirectoryReader(input_files=files)
    docs = reader.load_data()
    if verbose:
        print(f"Loaded {len(docs)} docs")

    parser = SentenceSplitter()
    nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

    if verbose:
        print(f"Parsed {len(nodes)} nodes")

    return nodes

In [ ]:
doc_nodes = load_corpus(docs_paths, verbose=True)

Loading files ['/content/drive/MyDrive/RangerAI/Documents/RFQ_2024-34_Supply_and_Delivery_of_Road_Salt__Sodium_Chloride__Final.pdf', '/content/drive/MyDrive/RangerAI/Documents/RFQ_2024-034_-_Surrey_Sport_and_Leisure_Complex_-_Child_Minding_Play_Space_and_Multi_Purpose_Room_Renovation.pdf', '/content/drive/MyDrive/RangerAI/Documents/RFQ_2024-021_-_Supply_and_Delivery__Lamps_and_Ballasts.pdf', '/content/drive/MyDrive/RangerAI/Documents/RFT_21098_-_Supply_of_Stand-alone_Sandblast_Cabinet_.pdf', '/content/drive/MyDrive/RangerAI/Documents/D785-01_ITT.pdf', '/content/drive/MyDrive/RangerAI/Documents/E622-00_ITT_VCH_VGH_JPN_General_Radiology_DR_Room_Replace.pdf', '/content/drive/MyDrive/RangerAI/Documents/BCI-RFP-2024-09-16_-_Employee_Recognition_Platform_RFP.pdf', '/content/drive/MyDrive/RangerAI/Documents/RFP_20801_Supply_of_Wheeled_Rock_Drill.pdf', '/content/drive/MyDrive/RangerAI/Documents/RFP_2024_PWB_31_Colwood_Creek_Washroom_Building.pdf', '/content/drive/MyDrive/RangerAI/Documents/ITQ

Parsing nodes:   0%|          | 0/298 [00:00<?, ?it/s]

Parsed 304 nodes


## **Synthesizing Question-Answer Pairs from Documents**

In [ ]:
OPENAI_API_KEY = 'sk-...'
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
import json
from llama_index.finetuning import generate_qa_embedding_pairs

In [ ]:
from llama_index.llms.openai import OpenAI

qa_embedding = generate_qa_embedding_pairs(
    llm=OpenAI(model="gpt-4o-mini"), nodes=doc_nodes
)

qa_embedding.save_json("/content/drive/MyDrive/RangerAI/QA/qa_pairs.json")

304it [05:20,  1.08s/it]


Final dataset saved.


## **Loading Question-Answer Pairs** (optional)

In [ ]:
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

qa_pairs = EmbeddingQAFinetuneDataset.from_json("/content/drive/MyDrive/RangerAI/QA/qa_pairs.json")

## **Loading Embedding Model from Hugging Face**

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## **Building Index**

In [ ]:
from llama_index.core.schema import TextNode
from llama_index.core import VectorStoreIndex

In [ ]:
corpus = qa_pairs.corpus
nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()]
index = VectorStoreIndex(nodes, embed_model=embed_model)

## **Storing Index**

In [ ]:
index.storage_context.persist(persist_dir="/content/drive/MyDrive/RangerAI/index")

## **Loading Index**

In [ ]:
from llama_index.core.storage import StorageContext
from llama_index.core import load_index_from_storage

In [ ]:
storage_context = StorageContext.from_defaults(persist_dir="/content/drive/MyDrive/RangerAI/index")
index = load_index_from_storage(storage_context, embed_model=embed_model)

## **Generating Response to Queries**

In [ ]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core import PromptTemplate

In [ ]:
from IPython.display import Markdown, display

In [53]:
retriever = index.as_retriever(similarity_top_k=10)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=get_response_synthesizer(),
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

qa_prompt_str = """
Persona: You are an AI assistant designed to provide comprehensive and concise answers to user questions about Requests for Proposals (RFPs) and Requests for Quotations (RFQs).

Task: Based on the provided Context, answer the user's questions in a clear and informative manner. Use bullet points to organize your responses and ensure clarity.

Context: {context_str}
Query: {query_str}
Answer:
"""

qa_prompt = PromptTemplate(qa_prompt_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt}
)


In [54]:
def get_response(query):
    response = query_engine.query(query)
    display(Markdown(str(response)))

In [55]:
get_response('List available RFQs.')

- RFQ 1220-040-2024-021 - Supply and Delivery, Lamps and Ballasts
- RFQ 2024-34 Supply and Delivery of Road Salt (Sodium Chloride)
- RFQ 1220-040-2024-034 - Surrey Sport and Leisure Complex - Child Minding Play Space and Multi-Purpose Room Renovation

Please let me know if you need more information on any specific RFQ.

In [56]:
get_response('what are key points of RFQ Supply and Delivery of Road Salt? ')

- The RFQ is for the Supply and Delivery of Road Salt (Sodium Chloride) by the City of Dawson Creek.
- The successful bidder will be responsible for supplying up to 500 tonnes of bulk winter road salt per year meeting specific specifications.
- The bidder must also provide all necessary tools, equipment, personnel, and fuel for depositing the material on City property.
- Deliveries must be a minimum of 45 tonnes per delivery, primarily during winter months or as requested by the City.
- Deliveries must be made with a minimum of three days notice and according to a schedule set by the City.
- The delivery location is the City Public Works Yard in Dawson Creek, BC, unless an alternate site is identified.
- Bidders can request a debriefing after the procurement process outcome notification within 30 days.
- The RFQ process is governed by the laws of British Columbia and federal laws of Canada.
- The contract term is for three years with the option to renew for two additional one-year periods.
- The RFQ submission deadline is September 30, 2024, at 2:00 p.m. MST, with the anticipated award of the contract on October 21, 2024.

In [65]:
get_response('As for the RFQ Supply and Delivery of Road Salt, When are proposals due, when will contracts be awarded, and what are the project milestones?')

- Proposals for the RFQ Supply and Delivery of Road Salt are due on September 30, 2024, at 2:00 p.m. MST.
- The anticipated award of the contract is scheduled for October 21, 2024.
- The project milestones include the commencement of the contract on November 1, 2024, and its conclusion on October 31, 2027, with the option to renew for two additional one-year periods.

In [71]:
get_response('what are key points of the ITQ Supply and delivery of One (1) Half Ton Pickup Truck?')

- The ITQ is for the supply and delivery of One (1) Half Ton Pickup Truck for the Oak Bay Fire Department.
- The ITQ Number is OBFD 27-2024.
- The ITQ Closing Time is October 17, 2024, at 2:00 pm local time.
- Quotations must be submitted in accordance with the requirements of Section 6 of the ITQ.
- The Quotation Form in Appendix B must be completed and submitted by the vendor.
- The Quotation should include the total price listed in the Quotation plus applicable PST/GST.
- The Quotation must be open for acceptance for a specified number of days following the Closing Time.
- Acceptance of the offer will create a contract as described in the purchase order, Quotation, Product specifications, and Terms and Conditions.
- The Quotation should be submitted in response to ITQ # OBFD 27-2024 for the Supply and delivery of One (1) Half Ton Pickup Truck.